#### 이번 대회에서는 범주형 변수 전처리를 위해 One-Hot Encoding과 for문을 사용했습니다.

#### 이는 train data로 fit한 One-Hot Encoder로 test data를 transform할 경우,
#### train data에는 속하지 않은 데이터가 test data에 있을 가능성이 있어 에러가 발생할 수 있기 때문입니다.

#### 이를 방지하기 위해 예외적인 상황에 대처할 수 있는 코드를 삽입해서 이중 for문을 작성했습니다.
#### 참고해 주시길 바랍니다.

# Import

In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge

# Fixed RandomSeed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Data Load

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [46]:
!unzip -qq "/content/gdrive/MyDrive/브동산.zip"

replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [55]:
train_df = pd.read_csv('/content/train.csv')

In [56]:
train_x = train_df.drop(columns=['ID', 'monthlyRent(us_dollar)'])
train_y = train_df['monthlyRent(us_dollar)']

In [57]:
test_x =  pd.read_csv('/content/test.csv').drop(columns=['ID'])

# Data Pre-processing

#### 범주형 변수는 크게 명목형 변수와 순서형 변수로 나눌 수 있습니다.
#### 순서형 변수의 경우 그 순서대로 수치값을 레이블로 부여하여 간단히 수치화 할 수 있지만,
#### 명목형 변수의 경우 값들의 순서 관계가 없어 수치 레이블링으로는 그 관계를 정확히 표현할 수 없습니다.
#### 그렇기에 명목형 변수의 경우 값들 각각을 새로운 컬럼으로 만들고, 원래 해당하던 값에는 1을,
#### 아닐 경우 0을 부여하는 One-Hot Encoding 방법이 존재합니다. 

In [58]:
# qualitative column one-hot encoding
qual_col = ['propertyType','suburbName']
ohe = OneHotEncoder(sparse=False)

for i in qual_col:
    train_x = pd.concat([train_x, pd.DataFrame(ohe.fit_transform(train_x[[i]]), columns=ohe.categories_[0])], axis=1)
    
    for qual_value in np.unique(test_x[i]): 
        if qual_value not in np.unique(ohe.categories_): 
            ohe.categories_ = np.append(ohe.categories_, qual_value)
    # One Hot Encoder가 Test 데이터로부터 Fitting되는 것은 Data Leakage이므로, Test 데이터에는 Train 데이터로 Fitting된 One Hot Encoder로부터 transform만 수행되어야 합니다.
    test_x = pd.concat([test_x, pd.DataFrame(ohe.transform(test_x[[i]]), columns=ohe.categories_[0])], axis=1)
    
train_x = train_x.drop(qual_col, axis=1)
test_x = test_x.drop(qual_col, axis=1)
print('Done.')

Done.


In [59]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
import lightgbm as lgb

In [63]:
train_ds = lgb.Dataset(train_x, label = train_y)
test_ds = lgb.Dataset(train_x)

In [64]:
params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

In [67]:
model = lgb.LGBMRegressor()

In [68]:
model.fit(train_x,train_y)

LGBMRegressor()

# Model Fit

In [69]:
predictions = model.predict(test_x)

# Prediction

# Submit

In [70]:
submission = pd.read_csv('/content/sample_submission.csv')

In [71]:
submission

,ID,monthlyRent(us_dollar)
0,Test_0000,0
1,Test_0001,0
2,Test_0002,0
3,Test_0003,0
4,Test_0004,0
...,...,...
8688,Test_8688,0
8689,Test_8689,0
8690,Test_8690,0
8691,Test_8691,0


In [72]:
submission['monthlyRent(us_dollar)'] = predictions

In [73]:
submission

,ID,monthlyRent(us_dollar)
0,Test_0000,187.828218
1,Test_0001,228.109828
2,Test_0002,327.176275
3,Test_0003,177.864902
4,Test_0004,91.957808
...,...,...
8688,Test_8688,363.726531
8689,Test_8689,349.766193
8690,Test_8690,204.229291
8691,Test_8691,159.337724


In [74]:
submission.to_csv('./submit.csv', index=False)